In [ ]:
import os

# if you want to use the GPU
#device = 'gpu'
#os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=' + device + ',floatX=float32'

import argparse
import csv
import datetime
import glob
import math
import sys
import time
import cv2
import random
import pickle
import itertools

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity

#import theano
from theano import config

import keras
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers import GRU, LSTM, CuDNNGRU, CuDNNLSTM, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.models import load_model
import h5py

from IPython.display import display


np.random.seed(123)


# %% Imports
from pathlib import Path
import sys
root_dir = Path().resolve()
sys.path.append(str(root_dir / 'src'))

from recsys_common import *

# Get Data

In [ ]:
meta=get_metadata()

In [ ]:
sessions=get_sessions(True,.05,True,0.25)
sessions['impressions']=sessions['impressions'].str.split('\\|')
sessions['prices']=sessions['prices'].str.split('\\|')
print(sessions.dtypes)
sessions.head()

## Get Splits

In [ ]:
test = sessions.loc[sessions.is_train==False].reset_index(drop=True)
# train = sessions.loc[(sessions.is_train==True)]
train = sessions.loc[(sessions.is_train==True) & (sessions.is_validation==False)].reset_index(drop=True)
dev = sessions.loc[(sessions.is_train==True) & (sessions.is_validation==True)].reset_index(drop=True)

print('test',test.shape)
print('train',train.shape)
print('dev',dev.shape)

sessions.loc[(sessions.is_validation==True)].reset_index(drop=True)

# Create Train and Validation sets

## Create Training set in right format

In [ ]:
train_1 = train[['user_id', 'session_id', 'timestamp','step', 'action_type', 'reference', 'impressions', 'prices']]
#train_2 = train_1.copy()
train_1['key'] = train_1['reference']
train_1=train_1.set_index('key')
meta['key'] = meta['item_id'].astype(str)
meta = meta.set_index('key')
train_1 = train_1.join(meta, on='key')
train_1['key1'] = (train_1['user_id'] + '_' + train_1['session_id'] + '_' + train_1['timestamp'].astype(str) + '_' + train_1['step'].astype(str))
train_1 = train_1.set_index(['key1', 'step'])
train_1 = train_1.drop(train_1.columns[[0, 1, 2, 3, 4, 5, 6, 7]], axis=1) 
#train_4 = train_4.set_index('session_id')
train_1.head(30)

In [ ]:
train_1.shape

### delete nan

In [ ]:
train_1 = train_1.dropna()
train_1.head()

In [ ]:
train_1.shape

### transform into 3D Tensor

In [ ]:
train1 = train_1.to_xarray()

### Save as pickle

In [ ]:
pickle_out = open("X_train.pickle","wb")
pickle.dump(train1, pickle_out)
pickle_out.close()

### Create Y_train

In [ ]:
df = train.copy()

A=[]
keys=df['action_type']
for i in keys:
    if i == 'clickout item':
        A.append(1)
    else:
        A.append('NaN')

y_train = pd.DataFrame(A)     
y_train.head()


In [ ]:
y_train1 = train_1.join(y_train)

In [ ]:
y_train1 = y_train1.dropna()
y_train1.head()

### save as pickle

In [ ]:
pickle_out = open("y_train.pickle","wb")
pickle.dump(y_train1, pickle_out)
pickle_out.close()

## Create Validation set in right format

In [ ]:
test_1 = test[['user_id', 'session_id', 'timestamp','step', 'action_type', 'reference', 'impressions', 'prices']]
test_1['key'] = test_1['reference']
test_1 = test_1.set_index('key')
meta['key'] = meta['item_id'].astype(str)
meta = meta.set_index('key')
test_1 = train_1.join(meta, on='key')
test_1['key1'] = (test_1['user_id'] + '_' + test_1['session_id'] + '_' + test_1['timestamp'].astype(str) + '_' + test_1['step'].astype(str))
test_1 = test_1.set_index(['key1', 'step'])
test_1 = test_1.drop(test_1.columns[[0, 1, 2, 3, 4, 5, 6, 7]], axis=1) 

test_1.head(30)

### delete nan

In [ ]:
test_1 = test_1.dropna()
test_1.head()

### transform into 3D Tensor

In [ ]:
test1 = test_1.to_xarray()

### save as pickle

In [ ]:
# Training Set:

pickle_out = open("X_val.pickle","wb")
pickle.dump(test1, pickle_out)
pickle_out.close()

### Create Y_val

In [ ]:
df = test.copy()
A=[]
keys=df['action_type']
for i in keys:
    if i == 'clickout item':
        A.append(1)
    else:
        A.append('NaN')

y_test = pd.DataFrame(A)     
y_test.head()

In [ ]:
y_test1 = test_1.join(y_test)

In [ ]:
y_test1 = y_test1.dropna()
y_test1.head()

### save as pickle

In [ ]:
pickle_out = open("y_val.pickle","wb")
pickle.dump(y_test1, pickle_out)
pickle_out.close()

## get metadata similarities of of each hotel 

In [ ]:
sim = cosine_similarity(meta)

In [ ]:
pickle_out = open("similarities.pickle","wb")
pickle.dump(sim, pickle_out)
pickle_out.close()